# 🎨 Notebook 05: Unconditional Protein Generation

**Learning Objective**: Generate novel protein backbones from scratch using diffusion models

## 💻 GPU Requirements

**⚠️ GPU Optional but Recommended**
- Works on CPU but generation will be slow (10-30 minutes per protein)
- With GPU: 30 seconds - 2 minutes per protein
- Recommended: T4 GPU or better (available free on Google Colab)

**Running on Google Colab**:
1. Runtime → Change runtime type → T4 GPU
2. See [colab_gpu_test.ipynb](../../colab_gpu_test.ipynb) to verify GPU is working

---

## 📚 What You'll Learn

1. Complete RFDiffusion sampling loop
2. GPU-accelerated protein generation
3. Noise schedules and timestep selection
4. Post-processing generated structures
5. Visualization of generation process

## 🔧 Setup and GPU Detection

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation
import time

# Detect and setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  Running on CPU - generation will be slower")
    print("For faster generation, enable GPU in Runtime → Change runtime type")

## 🏗️ Build Simplified RFDiffusion Model

We'll implement a simplified version that captures the key ideas:
- SE(3) equivariant updates to rigid body frames
- Diffusion process on backbone coordinates
- GPU-accelerated inference

In [ ]:
class SimplifiedRFDiffusion(nn.Module):
    """Simplified RFDiffusion model for educational purposes."""
    
    def __init__(self, hidden_dim=128, num_layers=4):
        super().__init__()
        self.hidden_dim = hidden_dim
        
        # Embed timestep
        self.time_embed = nn.Sequential(
            nn.Linear(1, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # Process per-residue features
        self.coord_embed = nn.Linear(3, hidden_dim)
        
        # Simple attention-based layers (simplified IPA)
        self.layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=hidden_dim,
                nhead=4,
                dim_feedforward=hidden_dim*4,
                batch_first=True
            )
            for _ in range(num_layers)
        ])
        
        # Predict coordinate updates
        self.coord_out = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 3)
        )
    
    def forward(self, coords, t):
        """
        Args:
            coords: (batch, n_residues, 3) - Cα coordinates
            t: (batch,) - timestep (0 to 1, where 1 is pure noise)
        
        Returns:
            coord_updates: (batch, n_residues, 3) - predicted denoising updates
        """
        batch_size, n_res, _ = coords.shape
        
        # Embed timestep
        t_embed = self.time_embed(t.view(-1, 1))  # (batch, hidden_dim)
        t_embed = t_embed.unsqueeze(1).expand(-1, n_res, -1)  # (batch, n_res, hidden_dim)
        
        # Embed coordinates
        coord_feat = self.coord_embed(coords)  # (batch, n_res, hidden_dim)
        
        # Combine features
        x = coord_feat + t_embed
        
        # Apply transformer layers
        for layer in self.layers:
            x = layer(x)
        
        # Predict coordinate updates
        coord_updates = self.coord_out(x)
        
        return coord_updates

# Initialize model and move to GPU
model = SimplifiedRFDiffusion(hidden_dim=128, num_layers=4).to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Model on device: {next(model.parameters()).device}")

## 🎲 Diffusion Process Implementation

In [ ]:
def cosine_beta_schedule(timesteps, s=0.008):
    """Cosine schedule as proposed in Improved DDPM."""
    steps = timesteps + 1
    x = torch.linspace(0, timesteps, steps)
    alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * np.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return torch.clip(betas, 0.0001, 0.9999)

class DiffusionProcess:
    """Handles forward and reverse diffusion."""
    
    def __init__(self, num_timesteps=100, device='cpu'):
        self.num_timesteps = num_timesteps
        self.device = device
        
        # Noise schedule
        self.betas = cosine_beta_schedule(num_timesteps).to(device)
        self.alphas = 1.0 - self.betas
        self.alphas_cumprod = torch.cumprod(self.alphas, dim=0)
        
        print(f"Diffusion schedule on device: {self.alphas_cumprod.device}")
    
    def add_noise(self, x0, t, noise=None):
        """
        Forward process: q(x_t | x_0)
        
        Args:
            x0: (batch, n_res, 3) - clean coordinates
            t: (batch,) - timestep indices (0 to num_timesteps-1)
            noise: (batch, n_res, 3) - optional pre-generated noise
        
        Returns:
            xt: noisy coordinates
            noise: the noise that was added
        """
        if noise is None:
            noise = torch.randn_like(x0)
        
        # Get alpha values for timesteps
        alpha_t = self.alphas_cumprod[t].view(-1, 1, 1)  # (batch, 1, 1)
        
        # q(x_t | x_0) = sqrt(alpha_t) * x_0 + sqrt(1 - alpha_t) * noise
        xt = torch.sqrt(alpha_t) * x0 + torch.sqrt(1 - alpha_t) * noise
        
        return xt, noise
    
    @torch.no_grad()
    def denoise_step(self, model, xt, t_idx):
        """
        Single reverse diffusion step: p(x_{t-1} | x_t)
        
        Args:
            model: denoising model
            xt: (batch, n_res, 3) - noisy coordinates at timestep t
            t_idx: integer timestep index
        
        Returns:
            x_prev: coordinates at timestep t-1
        """
        batch_size = xt.shape[0]
        
        # Timestep as continuous value (0 to 1)
        t = torch.full((batch_size,), t_idx / self.num_timesteps, device=self.device)
        
        # Predict noise
        predicted_noise = model(xt, t)
        
        # Compute x_0 prediction
        alpha_t = self.alphas_cumprod[t_idx]
        x0_pred = (xt - torch.sqrt(1 - alpha_t) * predicted_noise) / torch.sqrt(alpha_t)
        
        # Compute x_{t-1}
        if t_idx > 0:
            alpha_prev = self.alphas_cumprod[t_idx - 1]
            beta_t = self.betas[t_idx]
            
            # Posterior mean
            x_prev = torch.sqrt(alpha_prev) * x0_pred + \
                     torch.sqrt(1 - alpha_prev - beta_t) * predicted_noise
            
            # Add noise (except at last step)
            noise = torch.randn_like(xt) * torch.sqrt(beta_t)
            x_prev = x_prev + noise
        else:
            x_prev = x0_pred
        
        return x_prev

# Initialize diffusion process
diffusion = DiffusionProcess(num_timesteps=100, device=device)
print(f"✅ Diffusion process ready with {diffusion.num_timesteps} timesteps")

## 🎨 Generate Protein Backbone (GPU Accelerated!)

Now let's generate a protein from pure noise. This will run on GPU if available.

In [ ]:
@torch.no_grad()
def generate_protein(model, diffusion, n_residues=50, device='cpu'):
    """
    Generate a protein backbone from pure noise.
    
    Args:
        model: trained (or untrained for demo) diffusion model
        diffusion: DiffusionProcess instance
        n_residues: number of residues to generate
        device: 'cuda' or 'cpu'
    
    Returns:
        coords: (n_residues, 3) final generated Cα coordinates
        trajectory: list of intermediate coordinates for visualization
    """
    model.eval()
    
    # Start from pure noise
    xt = torch.randn(1, n_residues, 3, device=device) * 10.0  # Scale for protein-like distances
    
    trajectory = [xt[0].cpu().numpy()]
    
    print(f"Generating {n_residues}-residue protein...")
    start_time = time.time()
    
    # Reverse diffusion
    for t in range(diffusion.num_timesteps - 1, -1, -1):
        if t % 20 == 0:
            print(f"  Step {diffusion.num_timesteps - t}/{diffusion.num_timesteps}")
        
        xt = diffusion.denoise_step(model, xt, t)
        
        # Save trajectory snapshots
        if t % 10 == 0:
            trajectory.append(xt[0].cpu().numpy())
    
    elapsed = time.time() - start_time
    print(f"✅ Generation complete in {elapsed:.2f}s")
    print(f"   ({elapsed/n_residues:.3f}s per residue)")
    
    coords = xt[0].cpu().numpy()
    
    return coords, trajectory

# Generate a protein!
n_residues = 30  # Start small for demo
coords_generated, trajectory = generate_protein(model, diffusion, n_residues, device=device)

print(f"\nGenerated coordinates shape: {coords_generated.shape}")
print(f"Coordinate range: [{coords_generated.min():.2f}, {coords_generated.max():.2f}]")

## 📊 Visualize Generated Protein

In [ ]:
def visualize_generation_process(trajectory):
    """Show how the protein emerges from noise."""
    n_snapshots = min(len(trajectory), 6)
    indices = np.linspace(0, len(trajectory)-1, n_snapshots, dtype=int)
    
    fig = plt.figure(figsize=(18, 3))
    
    for i, idx in enumerate(indices):
        coords = trajectory[idx]
        ax = fig.add_subplot(1, n_snapshots, i+1, projection='3d')
        
        ax.plot(coords[:, 0], coords[:, 1], coords[:, 2], 
                'o-', linewidth=2, markersize=6, alpha=0.7)
        
        step = (len(trajectory) - 1 - idx) * 10
        ax.set_title(f'Step {step}/{diffusion.num_timesteps}', fontweight='bold')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        
        # Set consistent scale
        ax.set_xlim(-15, 15)
        ax.set_ylim(-15, 15)
        ax.set_zlim(-15, 15)
    
    plt.tight_layout()
    plt.show()

# Visualize the generation process
visualize_generation_process(trajectory)

# Final structure
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

ax.plot(coords_generated[:, 0], coords_generated[:, 1], coords_generated[:, 2],
        'o-', linewidth=3, markersize=8, color='#2E86AB', alpha=0.8)

# Color by position (N-terminus to C-terminus)
colors = plt.cm.viridis(np.linspace(0, 1, len(coords_generated)))
ax.scatter(coords_generated[:, 0], coords_generated[:, 1], coords_generated[:, 2],
           c=colors, s=100, alpha=0.9, edgecolors='black', linewidth=0.5)

ax.set_xlabel('X (Å)', fontsize=12)
ax.set_ylabel('Y (Å)', fontsize=12)
ax.set_zlabel('Z (Å)', fontsize=12)
ax.set_title(f'Generated Protein ({n_residues} residues)', fontsize=14, fontweight='bold')

plt.show()

# Analyze distances
distances = np.linalg.norm(np.diff(coords_generated, axis=0), axis=1)
print(f"\nCα-Cα distances:")
print(f"  Mean: {distances.mean():.2f} Å (expected ~3.8 Å)")
print(f"  Std:  {distances.std():.2f} Å")
print(f"  Min:  {distances.min():.2f} Å")
print(f"  Max:  {distances.max():.2f} Å")

## 🔑 Key Takeaways

### What We Learned

1. **GPU Acceleration** ⚡
   - Model and data automatically move to GPU with `.to(device)`
   - Generation is 10-100x faster on GPU
   - All tensor operations happen on the same device

2. **Diffusion Sampling Loop**
   - Start from pure random noise
   - Iteratively denoise using the model
   - Each step predicts and removes a bit of noise

3. **Untrained Model Limitations**
   - This model wasn't trained, so output is still somewhat random
   - Real RFDiffusion is trained on thousands of protein structures
   - Training teaches realistic protein geometry and secondary structures

4. **Next Steps for Real Generation**
   - Train model on PDB structures
   - Add proper SE(3) equivariant layers (full IPA)
   - Include side-chain prediction
   - Add structural validity checks

---

## 🎯 Practice Exercises

1. **Experiment with generation parameters**:
   - Try different numbers of residues (20, 50, 100)
   - Modify noise schedules
   - Change number of denoising steps

2. **Benchmark GPU speedup**:
   - Time generation on CPU vs GPU
   - Try different batch sizes
   - Profile memory usage

3. **Improve the model**:
   - Add more layers
   - Increase hidden dimensions
   - Implement proper attention mechanisms

---

## ➡️ Next Notebook

**Notebook 06: Motif Scaffolding** - Design proteins around specific functional motifs by conditioning the generation process.

**Note**: To train this model properly, you would need:
- PDB protein structure dataset
- Training loop with loss computation
- Several hours of GPU time
- See advanced tutorials for full training pipeline